In [17]:
import pandas as pd
import tensorflow
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.estimator.inputs import input_fn_pandas



ImportError: cannot import name 'LSTM' from 'tensorflow.python.keras.layers' (/opt/conda/lib/python3.10/site-packages/tensorflow/python/keras/layers/__init__.py)

In [16]:
# !pip uninstall tensorflow -y
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.2 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successfully uninstalled tensorboard-2.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensor

In [ ]:
if __name__ == "__main__":
    data = pd.read_parquet('/kaggle/input/next-orders/Train (1).parquet')

#     sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = data.select_dtypes(include=['int', 'int8', 'int32', 'uint32', 'uint8', 'float']).drop(columns = ['user_id', 'target'], axis = 1).columns.tolist()

#     data[sparse_features] = data[sparse_features].fillna('-1', )
#     data[dense_features] = data[dense_features].fillna(0, )
    target = ['target']

#     # 1.Label Encoding for sparse features,and do simple Transformation for dense features
#     for feat in sparse_features:
#         lbe = LabelEncoder()
#         data[feat] = lbe.fit_transform(data[feat])
    mms = MinMaxScaler(feature_range=(0, 1))
    data[dense_features] = mms.fit_transform(data[dense_features])

    # 2.count #unique features for each sparse field,and record dense feature field name

#     dnn_feature_columns = []
#     linear_feature_columns = []

#     for i, feat in enumerate(sparse_features):
#         dnn_feature_columns.append(tf.feature_column.embedding_column(
#             tf.feature_column.categorical_column_with_identity(feat, data[feat].max() + 1), 4))
#         linear_feature_columns.append(tf.feature_column.categorical_column_with_identity(feat, data[feat].max() + 1))
#     for feat in dense_features:
#         dnn_feature_columns.append(tf.feature_column.numeric_column(feat))
#         linear_feature_columns.append(tf.feature_column.numeric_column(feat))

    # 3.generate input data for model

    train, test = train_test_split(data, test_size=0.2, random_state=2021)

    # Not setting default value for continuous feature. filled with mean.

    train_model_input = input_fn_pandas(train, dense_features, target, shuffle=True)
    test_model_input = input_fn_pandas(test, dense_features, None, shuffle=False)

    # 4.Define Model,train,predict and evaluate
    model = DeepFMEstimator(dense_features, dense_features, task='binary',
                            config=tf.estimator.RunConfig(tf_random_seed=2021))

    model.train(train_model_input)
    pred_ans_iter = model.predict(test_model_input)
    pred_ans = list(map(lambda x: x['pred'], pred_ans_iter))
    #
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))